-------------------------------------------------------------------------------

CHIBO 2024

An attempt to static calculate the forces in (x,y,z) at all the suspension pickup points for various roll/jounce effects

18.12.2024: Created

03.01.2025: Outboard points calc working, moved functions to susprog.py and added loop 

05.01.2025: Added jounce updater for the inner points and included in susprog.py

08.01.2025: Added trail & scrub radius calc in loop

09.01.2025: Rewrote roll centre code to correct errors in original

11.01.2025: Added in trail/scrub/KPI/Caster angle evaluation. Bug was present for a while which miscalculated RC via unsmooth rotation of inner points. Went away for unknown reason

-------------------------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import susprog                      # Python file containing the equations for the loop 

In [2]:
# Declare motions for evaluation

# Define maximum roll and jounce inputs
body_roll = -5  # Maximum roll angle in degrees
body_jounce = 0.00  # Maximum jounce in meters (5 cm = 0.05)
sim_vales = 20

# Generate roll and jounce motion inputs
roll_angles = np.linspace(0, np.radians(body_roll), sim_vales)  # 0 to max roll angle in 100 steps
jounce_displacements = np.linspace(0, body_jounce, sim_vales)  # 0 to max jounce in 100 steps

In [4]:
# Suspension pickup points
#        x      y    z
upper_a_arm = np.array([
    [0.0015, 0.5065, 0.315],  # Outer point (converted from mm to m)
    [-0.135, 0.240, 0.220],   # Inner leading point
    [0.180, 0.255, 0.255]     # Inner trailing point
])

lower_a_arm = np.array([
    [-0.0175, 0.519, 0.137],  # Outer point
    [-0.180, 0.165, 0.087],   # Inner leading point
    [0.180, 0.170, 0.087]     # Inner trailing point
])

tie_rod = np.array([
    [0.060, 0.515, 0.160],  # Outer point
    [0.070, 0.195, 0.100]   # Inner point
])

tire_cop = np.array([-0.005, 0.525, 0])  # Converted from mm to m
force_vector = np.array([50, 500, -2000.0])  # Force at the CoP (N)


In [5]:
# Calculate lengths for upper and lower A-arms and tie rod
upper_lengths = [
    np.linalg.norm(upper_a_arm[1] - upper_a_arm[0]),  # Upper outer to inner leading
    np.linalg.norm(upper_a_arm[2] - upper_a_arm[0]),  # Upper outer to inner trailing
    np.linalg.norm(upper_a_arm[0] - lower_a_arm[0])   # Upper outer to lower outer
]

lower_lengths = [
    np.linalg.norm(lower_a_arm[1] - lower_a_arm[0]),  # Lower outer to inner leading
    np.linalg.norm(lower_a_arm[2] - lower_a_arm[0]),  # Lower outer to inner trailing
    np.linalg.norm(lower_a_arm[0] - tire_cop)         # Lower outer to tire center of pressure
]

tie_length = [
    np.linalg.norm(tie_rod[1] - tie_rod[0]),          # Tie rod outer to inner leading
    np.linalg.norm(upper_a_arm[0] - tie_rod[0]),      # Tie rod outer to upper outer
    np.linalg.norm(lower_a_arm[0] - tie_rod[0])       # Tie rod outer to lower outer
]

In [7]:
def suspension_geometry_calc(upper_points, lower_points, cop):
    """
    Calculate trail, scrub, kingpin inclination, and camber angle.
    
    Args:
        upper_points: np.array, upper suspension points [outer, inner].
        lower_points: np.array, lower suspension points [outer, inner].
        cop: np.array, center of the contact patch (x, y, z).
    
    Returns:
        dict: Dictionary containing trail, scrub, kingpin inclination (degrees), and camber angle (degrees).
    """
    # Lower outer point
    p0 = lower_points[0, :]
    # Upper outer point
    p1 = upper_points[0, :]
    
    # Calculate the vector for kingpin axis
    kingpin_vector = p1 - p0
    
    # Trail and Scrub
    tz = -p0[2] / (p1[2] - p0[2])  # Intersection with the ground (z = 0)
    x0 = p0[0] + tz * (p1[0] - p0[0])
    y0 = p0[1] + tz * (p1[1] - p0[1])
    trail = x0 - cop[0]
    scrub = y0 - cop[1]
    
    # Kingpin Inclination (XZ-plane)
    kpi_vector_xz = np.array([kingpin_vector[0], kingpin_vector[2]])  # Project to XZ-plane
    kpi_angle = np.arctan2(kpi_vector_xz[0], kpi_vector_xz[1])  # Angle with vertical (z-axis)
    kpi_angle_deg = np.degrees(kpi_angle)
    
    # Camber Angle (YZ-plane)
    camber_vector_yz = np.array([kingpin_vector[1], kingpin_vector[2]])  # Project to YZ-plane
    camber_angle = np.arctan2(camber_vector_yz[0], camber_vector_yz[1])  # Angle with vertical (z-axis)
    camber_angle_deg = np.degrees(camber_angle)
    
    return {
        "trail": trail,
        "scrub": scrub,
        "kingpin_inclination_deg": kpi_angle_deg,
        "camber_angle_deg": camber_angle_deg,
    }

In [9]:
steering = suspension_geometry_calc(upper_a_arm, lower_a_arm, tire_cop)
print("Trail:", steering["trail"])
print("Scrub:", steering["scrub"])
print("Kingpin Inclination (deg):", steering["kingpin_inclination_deg"])
print("Camber Angle (deg):", steering["camber_angle_deg"])

Trail: -0.027123595505617982
Scrub: 0.0036207865168539355
Kingpin Inclination (deg): 6.092771657336068
Camber Angle (deg): -4.016985413901983


In [8]:
# Run simulation

# Columns: [roll_angle (rad), jounce (m), suspension_positions (1x9), rc_x, rc_y, rc_z]
results = np.zeros((len(roll_angles), 29))  # 100 rows, 36 columns
#rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)
rc_data = susprog.calculate_rc_simple(upper_a_arm, lower_a_arm, tire_cop)
rc = rc_data[0:3]

# Set rotation point to origin
# rc = [0,0,0]

# Roll axis (example: x-axis)
roll_axis = np.array([1, 0, 0])

# Initialize rotated arms as a copy of the original
upper_modified = upper_a_arm.copy()
lower_modified = lower_a_arm.copy()
tierod_modified = tie_rod.copy()

# Loop through each roll and jounce input
for i, (roll_angle, jounce_disp) in enumerate(zip(roll_angles, jounce_displacements)):
    # Step 1: Rotate and jounce the inner points of the arms and tie rod
    upper_modified = susprog.rotate_points(upper_a_arm, rc, roll_axis, roll_angle, jounce_disp)
    #upper_modified = susprog.rotate_inner_points(upper_a_arm, rc, roll_angle)

    #upper_modified = susprog.jounce_innerpoints(upper_a_arm, upper_modified, jounce_disp)

    lower_modified = susprog.rotate_points(lower_a_arm, rc, roll_axis, roll_angle, jounce_disp)
    lower_modified = susprog.rotate_inner_points(lower_a_arm, rc, roll_angle)
    #lower_modified = susprog.jounce_innerpoints(lower_a_arm, lower_modified, jounce_disp)
    
    tierod_modified = susprog.rotate_points(tie_rod, rc, roll_axis, roll_angle, jounce_disp)
    # tierod_modified = susprog.rotate_inner_points(tie_rod, rc, roll_angle)
    #tierod_modified = susprog.jounce_innerpoints(tie_rod, tierod_modified, jounce_disp)

    # Step 3: Update outer points of lower, upper, and tie rod
    # Lower A-arm
    points_temp = np.vstack((lower_modified, tire_cop))
    lower_modified[0:1] = susprog.solve_outer_position(points_temp, lower_lengths)
    del points_temp

    # Upper A-arm
    points_temp = np.vstack((upper_modified, lower_modified[0:]))
    upper_modified[0:1] = susprog.solve_outer_position(points_temp, upper_lengths)
    del points_temp

    # Tie rod
    points_temp = np.vstack((tierod_modified, upper_modified[0:1], lower_modified[0:1]))
    tierod_modified[0:1] = susprog.solve_outer_position(points_temp, tie_length)
    del points_temp


    # Step 4: Calculate the new roll center
    rc_data = susprog.calculate_rc_simple(upper_modified, lower_modified, tire_cop)
    rc = rc_data[0:3]
    # Flatten geometry data dictionary into a list
    """geometry_values = [
        geo_roll_centre['instant_center_2d'][0],  # Instant center x
        geo_roll_centre['instant_center_2d'][1],  # Instant center z
        geo_roll_centre['swing_arm_length'],      # Swing arm length
        geo_roll_centre['upper_arm_angle'],       # Upper arm angle
        geo_roll_centre['lower_arm_angle'],       # Lower arm angle
        *geo_roll_centre['upper_normal'],         # Upper plane normal (3 components)
        *geo_roll_centre['lower_normal']          # Lower plane normal (3 components)
    ]"""
    
    steering_data = susprog.suspension_geometry_calc(upper_modified, lower_modified, tire_cop)
    steering_data = [
        steering_data['trail'],  # trail in x
        steering_data['scrub'],  # scrub in y
        steering_data['kingpin_inclination_deg'],      # KPI
        steering_data['camber_angle_deg'],       # Caster
    ]
    

    # Step 5: Store the results
    results[i, 0] = roll_angle  # Roll angle (in radians)
    results[i, 1] = jounce_disp  # Jounce displacement
    results[i, 2:11] = lower_modified.flatten()  # Lower A-arm positions (3x3 -> 1x9)
    results[i, 11:20] = upper_modified.flatten()  # Upper A-arm positions (3x3 -> 1x9)
    results[i, 20:25] = rc_data  # Roll center (1x8)
    #results[i, 23] = geo_roll_centre['instant_center_2d'][0]  # IC x
    #results[i, 24] = geo_roll_centre['instant_center_2d'][1]  # IC z
    #results[i, 25] = geo_roll_centre['swing_arm_length']      # Swing arm length
    #results[i, 26] = geo_roll_centre['upper_arm_angle']       # Upper arm angle
    #results[i, 27] = geo_roll_centre['lower_arm_angle']       # Lower arm angle
    #results[i, 28:31] = geo_roll_centre['upper_normal']       # Upper plane normal (x, y, z)
    #results[i, 31:34] = geo_roll_centre['lower_normal']       # Lower plane normal (x, y, z)
    results[i, 25:30] = steering_data # Steering axis==> tire cop (1x3)

del i

# Convert results to a DataFrame for easier visualization and saving
column_names = [
    "roll_angle", "jounce", 
    "lower_outer_x", "lower_outer_y", "lower_outer_z", 
    "lower_inner1_x", "lower_inner1_y", "lower_inner1_z", 
    "lower_inner2_x", "lower_inner2_y", "lower_inner2_z",
    "upper_outer_x", "upper_outer_y", "upper_outer_z", 
    "upper_inner1_x", "upper_inner1_y", "upper_inner1_z", 
    "upper_inner2_x", "upper_inner2_y", "upper_inner2_z",
    "rc_x", "rc_y", "rc_z",
    "ic_y", "ic_z", # "sal", "upper_angle", "lower_angle",  # Geometry data
    # "upper_normal_x", "upper_normal_y", "upper_normal_z",  # Upper plane normal
    # "lower_normal_x", "lower_normal_y", "lower_normal_z",   # Lower plane normal
    "Steering_trail_x", "Scrub_radius_y", "kingpin_inclination_deg", "camber_angle_deg"
]
results_df = pd.DataFrame(results, columns=column_names)

# Save the results as a CSV file (optional)
results_df.to_csv("suspension_simulation_results.csv", index=False)

# Display the first few rows
#print(results_df.head())


In [12]:
# Extract the initial and final suspension points from the results
lower_initial = results[0, 2:11].reshape(3, 3)  # First row, lower A-arm (3x3)
upper_initial = results[0, 11:20].reshape(3, 3)  # First row, upper A-arm (3x3)

lower_final = results[-1, 2:11].reshape(3, 3)  # Last row, lower A-arm (3x3)
upper_final = results[-1, 11:20].reshape(3, 3)  # Last row, upper A-arm (3x3)

# Extract roll centers
rc_initial = results[0, 20:23]  # Initial roll center
rc_final = results[-1, 20:23]  # Final roll center

# Create a 3D scatter plot
fig = go.Figure()

# Add initial lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_initial[:, 0], y=lower_initial[:, 1], z=lower_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='blue'),
    name='Lower A-arm (Initial)'
))

# Add final lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_final[:, 0], y=lower_final[:, 1], z=lower_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='red'),
    name='Lower A-arm (Final)'
))

# Add initial upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_initial[:, 0], y=upper_initial[:, 1], z=upper_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='green'),
    name='Upper A-arm (Initial)'
))

# Add final upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_final[:, 0], y=upper_final[:, 1], z=upper_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='orange'),
    name='Upper A-arm (Final)'
))

# Add roll center points
fig.add_trace(go.Scatter3d(
    x=[rc_initial[0], rc_final[0]],
    y=[rc_initial[1], rc_final[1]],
    z=[rc_initial[2], rc_final[2]],
    mode='markers+lines',
    marker=dict(size=8, color='purple'),
    name='Roll Center'
))

# Configure the layout
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis (m)',
        yaxis_title='Y-axis (m)',
        zaxis_title='Z-axis (m)'
    ),
    title="3D Suspension Geometry (Initial vs. Final)",
    legend=dict(x=0.1, y=0.9)
)

# Show the figure
fig.show()

# Print the initial and final points
print("Initial Lower A-arm Points:\n", lower_initial)
print("Final Lower A-arm Points:\n", lower_final)

print("Initial Upper A-arm Points:\n", upper_initial)
print("Final Upper A-arm Points:\n", upper_final)

print("Initial Roll Center:", rc_initial)
print("Final Roll Center:", rc_final)

Initial Lower A-arm Points:
 [[-0.0175  0.519   0.137 ]
 [-0.18    0.165   0.087 ]
 [ 0.18    0.17    0.087 ]]
Final Lower A-arm Points:
 [[-0.01743833  0.52260926  0.13710552]
 [-0.18        0.17109302  0.07232586]
 [ 0.18        0.176074    0.07189008]]
Initial Upper A-arm Points:
 [[ 0.0015  0.5065  0.315 ]
 [-0.135   0.24    0.22  ]
 [ 0.18    0.255   0.255 ]]
Final Upper A-arm Points:
 [[-0.00261908  0.51688376  0.31583911]
 [-0.135       0.25739934  0.19828308]
 [ 0.18        0.27539271  0.23184255]]
Initial Roll Center: [0.         0.         0.01061696]
Final Roll Center: [0.         0.         0.00989877]


In [9]:
# Plot RC vs roll

fig = go.Figure()

# Add the trace for "rc_z"
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Roll Centre Location, Z'
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Roll Angle (radians)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [16]:
# Plot RC vs jounce

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["jounce"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, z',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Jounce (m)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()


In [10]:
# Plot inner positions vs roll

# Create the plot
fig = go.Figure()

# Add Z heights vs roll to the primary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["lower_inner1_z"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["upper_inner1_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Upper Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add Y positions vs roll to the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["lower_inner1_y"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["upper_inner1_y"],
        mode='lines',
        marker=dict(color='green'),
        name='Upper Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='Z Heights (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Y Positions (m)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [11]:
# Plot inner positions vs roll

# Create the plot
fig = go.Figure()

# Add king pin incl
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["kingpin_inclination_deg"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add caster angle 
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["camber_angle_deg"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='KPI [°]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Caster angle [°]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()